# Cassava Classification - PyTorch Starter (Train)

## Library imports

In [1]:
import os
import numpy as np
import pandas as pd
import random

import albumentations as A
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim

from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from albumentations.pytorch import ToTensorV2

from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, StratifiedKFold

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings  
warnings.filterwarnings('ignore')

## Config files

In [2]:
pipeline = {'train' : True, 'lr_find' : False, 'validate' : False, 'test' : False, 'DEBUG' : False}
model_cfg = {'model_architecture': 'resnet18', 'model_name': 'R18_imagenet_v2',
             'init_lr': 1e-4, 'weight_path': ''}

train_cfg = {'batch_size': 16, 'shuffle': False, 'num_workers': 4, 'checkpt_every' : 1 }
valid_cfg = {'batch_size': 16, 'shuffle': False, 'num_workers': 4, 'validate_every' : 1 }
test_cfg  = {'batch_size': 16, 'shuffle': False, 'num_workers': 4}

DIR_INPUT = '../input/cassava-leaf-disease-classification'
SEED = 42
N_FOLDS = 5 #if pipeline['DEBUG'] else 5
N_EPOCHS = 5 #if pipeline['DEBUG'] else 10
BATCH_SIZE = 32
SIZE = 256

In [3]:
index_label_map = {
                0: "Cassava Bacterial Blight (CBB)", 
                1: "Cassava Brown Streak Disease (CBSD)",
                2: "Cassava Green Mottle (CGM)", 
                3: "Cassava Mosaic Disease (CMD)", 
                4: "Healthy"
                }

## Helper functions

In [4]:
def find_no_of_trainable_params(model):
    total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    #print(total_trainable_params)
    return total_trainable_params

In [5]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
#RANDOM_STATE = 42
set_seed(SEED)

## Dataset 

In [6]:
train_df = pd.read_csv(f'{DIR_INPUT}/train.csv')
train_df[['cls0', 'cls1', 'cls2', 'cls3', 'cls4']] = train_labels = pd.get_dummies(train_df.iloc[:, 1])

# For debugging.
if pipeline['DEBUG']:
    train_df = train_df.sample(n=200)
    train_df.reset_index(drop=True, inplace=True)

train_labels = train_df.iloc[:, 1].values

train_df.head()

,image_id,label,cls0,cls1,cls2,cls3,cls4
0,1000015157.jpg,0,1,0,0,0,0
1,1000201771.jpg,3,0,0,0,1,0
2,100042118.jpg,1,0,1,0,0,0
3,1000723321.jpg,1,0,1,0,0,0
4,1000812911.jpg,3,0,0,0,1,0


In [7]:
train_df.shape

(21397, 7)

In [8]:
class CassavaDataset(Dataset):
    
    def __init__(self, df, dataset='train', transforms=None):
    
        self.df = df
        self.transforms=transforms
        self.dataset=dataset
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        image_src = f'{DIR_INPUT}/{self.dataset}_images/{self.df.loc[idx, "image_id"]}'
        # print(image_src)
        image = cv2.imread(image_src, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # label = self.df.loc[idx, 'label']
        labels = self.df.loc[idx, ['cls0', 'cls1', 'cls2', 'cls3', 'cls4']].values
        labels = torch.from_numpy(labels.astype(np.int8))
        labels = labels.unsqueeze(-1)
        
        if self.transforms:
            transformed = self.transforms(image=image)
            image = transformed['image']

        return image, labels

## Transforms for Augumentations

In [9]:
transforms_train = A.Compose([
    A.RandomResizedCrop(height=SIZE, width=SIZE, p=1.0),
    A.Flip(),
    A.ShiftScaleRotate(rotate_limit=1.0, p=0.8),
    A.Normalize(p=1.0),
    ToTensorV2(p=1.0),
])

transforms_valid = A.Compose([
    A.Resize(height=SIZE, width=SIZE, p=1.0),
    A.Normalize(p=1.0),
    ToTensorV2(p=1.0),
])

## CV strategy

In [10]:
folds = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
oof_preds = np.zeros((train_df.shape[0],))

## Model class

In [11]:
class CassavaModel(nn.Module):
    
    def __init__(self, num_classes=5):
        super().__init__()
        
        self.backbone = torchvision.models.resnet18(pretrained=True)
        
        in_features = self.backbone.fc.in_features

        self.logit = nn.Linear(in_features, num_classes)
        
    def forward(self, x):
        batch_size, C, H, W = x.shape
        
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)
        
        x = F.adaptive_avg_pool2d(x,1).reshape(batch_size,-1)
        x = F.dropout(x, 0.25, self.training)

        x = self.logit(x)

        return x

## Loss function

In [12]:
class DenseCrossEntropy(nn.Module):

    def __init__(self):
        super(DenseCrossEntropy, self).__init__()
        
        
    def forward(self, logits, labels):
        logits = logits.float()
        labels = labels.float()
        
        logprobs = F.log_softmax(logits, dim=-1)
        
        loss = -labels * logprobs
        loss = loss.sum(-1)

        return loss.mean()

In [13]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device('cpu')
criterion = DenseCrossEntropy()

In [14]:
device

device(type='cuda', index=0)

## Lr_find

In [15]:
def plot_lr_finder_results(lr_finder): 
    # Create subplot grid
    fig = make_subplots(rows=1, cols=2)
    # layout ={'title': 'Lr_finder_result'}
    
    # Create a line (trace) for the lr vs loss, gradient of loss
    trace0 = go.Scatter(x=lr_finder['log_lr'], y=lr_finder['smooth_loss'],name='log_lr vs smooth_loss')
    trace1 = go.Scatter(x=lr_finder['log_lr'], y=lr_finder['grad_loss'],name='log_lr vs loss gradient')

    # Add subplot trace & assign to each grid
    fig.add_trace(trace0, row=1, col=1);
    fig.add_trace(trace1, row=1, col=2);
    #iplot(fig, show_link=False)
    fig.write_html(model_cfg['model_name'] + '_lr_find.html');

In [16]:
def find_lr(data_loader, init_value = 1e-8, final_value=100.0, beta = 0.98, num_batches = 200):
    assert(num_batches > 0)
    mult = (final_value / init_value) ** (1/num_batches)
    lr = init_value
    optimizer.param_groups[0]['lr'] = lr
    batch_num = 0
    avg_loss = 0.0
    best_loss = 0.0
    smooth_losses = []
    raw_losses = []
    log_lrs = []
    dataloader_it = iter(data_loader)
    progress_bar = tqdm(range(num_batches))
        
    for idx in progress_bar:
        batch_num += 1
        try:
            inputs, labels = next(dataloader_it)
        except StopIteration:
            dataloader_it = iter(data_loader)
            inputs, labels = next(dataloader_it)

        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        
        
        # handle exception in criterion
        try:
            # Forward pass
            log_ps = model(inputs)
            loss = criterion(log_ps, labels.squeeze(-1))
        except:
            if len(smooth_losses) > 1:
                grad_loss = np.gradient(smooth_losses)
            else:
                grad_loss = 0.0
            lr_finder_results = {'log_lr':log_lrs, 'raw_loss':raw_losses, 
                                 'smooth_loss':smooth_losses, 'grad_loss': grad_loss}
            return lr_finder_results        
        
        #Compute the smoothed loss
        avg_loss = beta * avg_loss + (1-beta) *loss.item()
        smoothed_loss = avg_loss / (1 - beta**batch_num)
        
        #Stop if the loss is exploding
        if batch_num > 1 and smoothed_loss > 50 * best_loss:
            if len(smooth_losses) > 1:
                grad_loss = np.gradient(smooth_losses)
            else:
                grad_loss = 0.0
            lr_finder_results = {'log_lr':log_lrs, 'raw_loss':raw_losses, 
                                 'smooth_loss':smooth_losses, 'grad_loss': grad_loss}
            return lr_finder_results
        
        #Record the best loss
        if smoothed_loss < best_loss or batch_num==1:
            best_loss = smoothed_loss
        
        #Store the values
        raw_losses.append(loss.item())
        smooth_losses.append(smoothed_loss)
        log_lrs.append(math.log10(lr))
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # print info
        progress_bar.set_description(f"loss: {loss.item()},smoothed_loss: {smoothed_loss},lr : {lr}")

        #Update the lr for the next step
        lr *= mult
        optimizer.param_groups[0]['lr'] = lr
    
    grad_loss = np.gradient(smooth_losses)
    lr_finder_results = {'log_lr':log_lrs, 'raw_loss':raw_losses, 
                         'smooth_loss':smooth_losses, 'grad_loss': grad_loss}
    return lr_finder_results

## one fold train and validate function

In [17]:
def train_one_fold(i_fold, model, optimizer, dataloader_train, dataloader_valid):
    
    train_fold_results = []

    for epoch in range(N_EPOCHS):
        print('  Epoch {}/{}'.format(epoch + 1, N_EPOCHS))
        model.train()
        tr_loss = 0
        
        # training iterator
        tr_iterator = iter(dataloader_train)
        train_progress_bar = tqdm(range(len(dataloader_train)))
    
        for idx in train_progress_bar:
            try:
                images, labels = next(tr_iterator)
            except StopIteration:
                tr_iterator = iter(dataloader_train)
                images, labels = next(tr_iterator)

            images = images.to(device, dtype=torch.float)
            labels = labels.to(device, dtype=torch.float)
            
            # Forward pass
            outputs = model(images)

            # Backward pass
            loss = criterion(outputs, labels.squeeze(-1))                
            tr_loss += loss.item()
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            # print to console
            train_progress_bar.set_description(f"Train_loss: {tr_loss} loss(avg): {tr_loss/(idx+1)}")

        # Validate
        model.eval()
        val_loss = 0.0
        val_preds = None
        val_labels = None
        
        valid_iterator = iter(dataloader_valid)
        valid_progress_bar = tqdm(range(len(dataloader_valid)))

        for idx in valid_progress_bar:
            try:
                images, labels = next(valid_iterator)
            except StopIteration:
                tr_iterator = iter(dataloader_valid)
                images, labels = next(valid_iterator)

            if val_labels is None:
                val_labels = labels.clone().squeeze(-1)
            else:
                val_labels = torch.cat((val_labels, labels.squeeze(-1)), dim=0)

            images = images.to(device, dtype=torch.float)
            labels = labels.to(device, dtype=torch.float)

            with torch.no_grad():
                outputs = model(images)

                loss = criterion(outputs, labels.squeeze(-1))
                val_loss += loss.item()

                preds = torch.softmax(outputs, dim=1).data.cpu()

                if val_preds is None:
                    val_preds = preds
                else:
                    val_preds = torch.cat((val_preds, preds), dim=0)
            
            # print to console
            valid_progress_bar.set_description(f"val_loss: {val_loss} loss(avg): {val_loss/(idx+1)}")

        
        val_preds = torch.argmax(val_preds, dim=1)

        train_fold_results.append({
            'fold': i_fold,
            'epoch': epoch,
            'train_loss': tr_loss / len(dataloader_train),
            'valid_loss': val_loss / len(dataloader_valid),
            'valid_score': accuracy_score(torch.argmax(val_labels, dim=1), val_preds)
        })

    return val_preds, train_fold_results

In [18]:
submissions = None
train_results = []

for i_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df, train_labels)):
    print("Fold {}/{}".format(i_fold + 1, N_FOLDS))

    valid = train_df.iloc[valid_idx]
    valid.reset_index(drop=True, inplace=True)

    train = train_df.iloc[train_idx]
    train.reset_index(drop=True, inplace=True)    

    dataset_train = CassavaDataset(df=train, dataset='train', transforms=transforms_train)
    dataset_valid = CassavaDataset(df=valid, dataset='train', transforms=transforms_valid)

    dataloader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, num_workers=4, shuffle=True)
    dataloader_valid = DataLoader(dataset_valid, batch_size=BATCH_SIZE, num_workers=4, shuffle=False)

    model = CassavaModel(num_classes=5)
    model.to(device)

    plist = [{'params': model.parameters(), 'lr': 1e-4}]
    optimizer = optim.Adam(plist, lr=1e-4)
    
    val_preds, train_fold_results = train_one_fold(i_fold, model, optimizer, dataloader_train, dataloader_valid)
    oof_preds[valid_idx] = val_preds.numpy()
    
    train_results = train_results + train_fold_results
    
    torch.save({
        'fold': i_fold,
        'lr': optimizer.state_dict()["param_groups"][0]['lr'],
        'model_state_dict': model.cpu().state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        # 'scheduler_state_dict'
        # 'scaler_state_dict'
    }, f"model_state_fold_{i_fold}.pth")

print("{}-Folds CV score: {:.4f}".format(N_FOLDS, accuracy_score(train_labels, oof_preds)))

Fold 1/5


Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth



  Epoch 1/5



  Epoch 2/5



  Epoch 3/5


...


  Epoch 2/5



  Epoch 3/5



  Epoch 4/5



  Epoch 5/5



5-Folds CV score: 0.8375


## Show train history

In [19]:
train_results = pd.DataFrame(train_results)
train_results.head(10)

,fold,epoch,train_loss,valid_loss,valid_score
0,0,0,0.657196,0.542884,0.809346
1,0,1,0.534289,0.543814,0.813785
2,0,2,0.499588,0.475143,0.834112
3,0,3,0.480589,0.502217,0.830841
4,0,4,0.457458,0.459768,0.841121
5,1,0,0.678633,0.604318,0.785280
6,1,1,0.540860,0.496041,0.827103
7,1,2,0.504875,0.472815,0.831308
8,1,3,0.481522,0.460141,0.840654
9,1,4,0.474315,0.414079,0.859813


In [20]:
fig = make_subplots(rows=2, cols=1)

colors = [
    ('#d32f2f', '#ef5350'),
    ('#303f9f', '#5c6bc0'),
    ('#00796b', '#26a69a'),
    ('#fbc02d', '#ffeb3b'),
    ('#5d4037', '#8d6e63'),
]

for i in range(N_FOLDS):
    data = train_results[train_results['fold'] == i]

    fig.add_trace(go.Scatter(x=data['epoch'].values,
                             y=data['train_loss'].values,
                             mode='lines',
                             visible='legendonly' if i > 0 else True,
                             line=dict(color=colors[i][0], width=2),
                             name='Train loss - Fold #{}'.format(i)),
                 row=1, col=1)

    fig.add_trace(go.Scatter(x=data['epoch'],
                             y=data['valid_loss'].values,
                             mode='lines+markers',
                             visible='legendonly' if i > 0 else True,
                             line=dict(color=colors[i][1], width=2),
                             name='Valid loss - Fold #{}'.format(i)),
                 row=1, col=1)
    
    fig.add_trace(go.Scatter(x=data['epoch'].values,
                             y=data['valid_score'].values,
                             mode='lines+markers',
                             line=dict(color=colors[i][0], width=2),
                             name='Valid score - Fold #{}'.format(i),
                             showlegend=False),
                 row=2, col=1)

fig.update_layout({
  "annotations": [
    {
      "x": 0.225, 
      "y": 1.0, 
      "font": {"size": 16}, 
      "text": "Train / valid losses", 
      "xref": "paper", 
      "yref": "paper", 
      "xanchor": "center", 
      "yanchor": "bottom", 
      "showarrow": False
    }, 
    {
      "x": 0.775, 
      "y": 1.0, 
      "font": {"size": 16}, 
      "text": "Validation scores", 
      "xref": "paper", 
      "yref": "paper", 
      "xanchor": "center", 
      "yanchor": "bottom", 
      "showarrow": False
    }, 
  ]
})

fig.show()